In [71]:
import pandas as pd
import numpy as np

# Load community data

In [72]:
community_data = pd.read_pickle('community_data.pkl')

In [73]:
df_communities = community_data['Communities']

df_communities.shape

(1741, 2)

In [74]:
df_communities_filtered = community_data['Communities_filtered']

df_communities_filtered.shape

(264, 2)

In [75]:
community_user_data = pd.read_pickle('community_users_data.pkl')

df_community_users = community_user_data['Users']

df_community_users.shape

(484252, 2)

# Load services data

In [76]:
services_data = pd.read_pickle('shared_interest_detection_data_services.pkl')

df_services = services_data['services']

In [77]:
df_services.head()

,User_Id,Service_Id,Service_Name,Service_URL,UserName_On_Service,UserURL_On_Service
18,spaceastronautics,blog,Blog,http://en.wikipedia.org/wiki/Blog,NaN,http://space.gs
19,spaceastronautics,facebook,Facebook,http://www.facebook.com/,NaN,http://www.facebook.com/profile.php?id=2949246...
20,spaceastronautics,facebook,Facebook,http://www.facebook.com/,NaN,http://www.facebook.com/profile.php?id=1171781...
21,spaceastronautics,facebook,Facebook,http://www.facebook.com/,NaN,http://www.facebook.com/profile.php?id=1227261...
22,spaceastronautics,twitter,Twitter,http://twitter.com/,oceanechoes,http://twitter.com/oceanechoes


In [78]:
df_community_user_services = df_community_users.merge(df_services[['User_Id', 'Service_Id']], how='left', on='User_Id')

In [79]:
df_community_user_services.head()

,Community,User_Id,Service_Id
0,3,brendafranzo,NaN
1,3,njsinc4el,blog
2,3,njsinc4el,twitter
3,3,praguebob,twelveseconds
4,3,praguebob,blog


In [80]:
df_community_services = df_community_user_services.groupby(['Community', 'Service_Id']).count().reset_index().rename({ 'User_Id': 'Service_Count' }, axis=1).sort_values(['Community', 'Service_Count'], ascending=[True, False]).rename({ 'Service_Id': 'Service' }, axis = 1)

In [81]:
df_community_services.describe()

,Community,Service_Count
count,6581.000000,6581.000000
mean,650.433673,207.629084
std,544.488664,2023.896968
min,0.000000,1.000000
25%,114.000000,1.000000
50%,573.000000,1.000000
75%,1091.000000,2.000000
max,1740.000000,69272.000000


In [82]:
df_community_services

,Community,Service,Service_Count
4,0,blog,67135
54,0,twitter,66108
14,0,flickr,34208
59,0,youtube,26682
7,0,delicious,26002
...,...,...,...
6576,1739,pandora,1
6577,1739,tumblr,1
6578,1739,twitter,1
6579,1739,youtube,1


In [83]:
df_community_service_usage = df_community_services.groupby('Community')['Service'].count().reset_index().rename({ 'Service': 'Service_Count' }, axis=1)

In [84]:
df_community_service_usage.sort_values('Service_Count', ascending=[False])

,Community,Service_Count
0,0,61
4,4,61
1,1,61
2,2,61
3,3,61
...,...,...
978,1123,1
970,1114,1
966,1109,1
1420,1714,1


In [85]:
df_community_service_usage = df_community_service_usage.merge(df_communities, on='Community').rename({ 'Size': 'User_Count' }, axis=1)

In [86]:
df_community_service_usage.describe()

,Community,Service_Count,User_Count
count,1438.000000,1438.000000,1438.000000
mean,845.142559,4.576495,336.217663
std,497.973776,7.286420,4732.813750
min,0.000000,1.000000,2.000000
25%,417.500000,1.000000,2.000000
50%,838.000000,2.000000,2.000000
75%,1265.750000,5.000000,3.000000
max,1740.000000,61.000000,147071.000000


In [87]:
df_communities

,Community,Size
0,0,147071
1,1,46218
2,2,45581
3,3,44992
4,4,43015
...,...,...
1736,1736,2
1737,1737,2
1738,1738,2
1739,1739,2


In [88]:
df_community_top_services = df_community_services.groupby(['Community']).head(3)

In [89]:
df_community_top_services

,Community,Service,Service_Count
4,0,blog,67135
54,0,twitter,66108
14,0,flickr,34208
115,1,twitter,25669
65,1,blog,17974
...,...,...,...
6571,1738,youtube,1
6572,1739,blog,2
6573,1739,facebook,1
6574,1739,flickr,1


In [90]:
df_community_top_services_agg = df_community_top_services.groupby('Community').apply(lambda x: set(x.Service)).reset_index(name='Services')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6044\334173625.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_community_top_services_agg = df_community_top_services.groupby('Community').apply(lambda x: set(x.Service)).reset_index(name='Services')


In [91]:
df_community_top_services_agg

,Community,Services
0,0,"{twitter, flickr, blog}"
1,1,"{twitter, feed, blog}"
2,2,"{twitter, feed, blog}"
3,3,"{youtube, twitter, blog}"
4,4,"{twitter, feed, blog}"
...,...,...
1433,1735,"{diigo, delicious, digg}"
1434,1737,"{youtube, blog}"
1435,1738,"{youtube, twitter, blog}"
1436,1739,"{flickr, facebook, blog}"


# Cluster similar communities based on top services

In [92]:
filtered_community_ids = df_communities_filtered['Community']

In [93]:
filtered_community_ids

0        0
1        1
2        2
3        3
4        4
      ... 
259    259
260    260
261    261
262    262
263    263
Name: Community, Length: 264, dtype: int64

In [94]:
df_community_top_services_filtered = df_community_top_services[df_community_top_services['Community'].isin(filtered_community_ids)]

In [95]:
df_community_top_services_filtered

,Community,Service,Service_Count
4,0,blog,67135
54,0,twitter,66108
14,0,flickr,34208
115,1,twitter,25669
65,1,blog,17974
...,...,...,...
2260,261,googletalk,1
2261,261,picasa,1
2262,261,twitter,1
2264,263,flickr,1


In [96]:
df_community_top_services_filtered_pivot = pd.pivot_table(
    df_community_top_services_filtered,
    index='Community',
    columns='Service',
    aggfunc='count',
    fill_value=0
)

In [97]:
df_community_top_services_filtered_pivot

Service_Count                                                      \
Service          amazon ameba backtype blog brightkite delicious digg diigo   
Community                                                                     
0                     0     0        0    1          0         0    0     0   
1                     0     0        0    1          0         0    0     0   
2                     0     0        0    1          0         0    0     0   
3                     0     0        0    1          0         0    0     0   
4                     0     0        0    1          0         0    0     0   
...                 ...   ...      ...  ...        ...       ...  ...   ...   
257                   0     0        0    1          0         0    0     0   
259                   0     0        0    0          0         1    0     0   
260                   0     0        0    0          0         0    0     0   
261                   0     0        0    0          0         0    0     0   
263                   0     0        0    0          0         0    0     0   

                           ...                                                \
Service   disqus facebook  ... mixx netflix pandora photobucket picasa plurk   
Community                  ...                                                 
0              0        0  ...    0       0       0           0      0     0   
1              0        0  ...    0       0       0           0      0     0   
2              0        0  ...    0       0       0           0      0     0   
3              0        0  ...    0       0       0           0      0     0   
4              0        0  ...    0       0       0           0      0     0   
...          ...      ...  ...  ...     ...     ...         ...    ...   ...   
257            0        0  ...    0       0       0           0      0     0   
259            0        0  ...    0       0       0           0      0     0   
260            0        0  ...    0       0       0           0      0     0   
261            0        0  ...    0       0       0           0      1     0   
263            0        0  ...    0       0       0           0      0     0   

                                              
Service   stumbleupon tumblr twitter youtube  
Community                                     
0                   0      0       1       0  
1                   0      0       1       0  
2                   0      0       1       0  
3                   0      0       1       1  
4                   0      0       1       0  
...               ...    ...     ...     ...  
257                 0      0       0       1  
259                 0      0       0       0  
260                 0      0       0       1  
261                 0      0       1       0  
263                 0      0       0       0  

[228 rows x 30 columns]

Jaccard Similarity Matrix

In [98]:
from scipy.spatial.distance import pdist, squareform

In [99]:
jaccard_distances_vector = pdist(df_community_top_services_filtered_pivot.values, metric='jaccard')

In [100]:
jaccard_distances_vector

array([0.5       , 0.5       , 0.5       , ..., 0.75      , 0.66666667,
       0.75      ], shape=(25878,))

In [101]:
print(np.min(jaccard_distances_vector))
print(np.max(jaccard_distances_vector))

0.0
1.0


In [102]:
condensed_dist_matrix = squareform(jaccard_distances_vector)

In [103]:
from scipy.cluster.hierarchy import linkage
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import fcluster

Analyze distance threshold

In [104]:
Z = linkage(jaccard_distances_vector, method='complete')

merge_distances = Z[:, 2]

candidate_thresholds = np.unique(merge_distances)

candidate_thresholds

array([0.        , 0.33333333, 0.5       , 0.66666667, 0.75      ,
       0.8       , 1.        ])

In [105]:
for dt in candidate_thresholds:

    labels = fcluster(Z, dt, criterion='distance')
    
    num_clusters = len(np.unique(labels))
    
    if num_clusters >= 2:
        score = silhouette_score(condensed_dist_matrix, labels, metric='precomputed')

        print(f"Silhouette score: {score:.4f} for distance threshold: {dt} and clusters count: {num_clusters}")
    


Silhouette score: 0.6535 for distance threshold: 0.0 and clusters count: 107
Silhouette score: 0.6110 for distance threshold: 0.3333333333333333 and clusters count: 94
Silhouette score: 0.4665 for distance threshold: 0.5 and clusters count: 44
Silhouette score: 0.4153 for distance threshold: 0.6666666666666666 and clusters count: 39
Silhouette score: 0.3761 for distance threshold: 0.75 and clusters count: 34
Silhouette score: 0.2409 for distance threshold: 0.8 and clusters count: 10


Apply agglomerative clustering

In [106]:
chosen_k = 94 # considering K from slightly reduced silhoutte score for meaningful clusters

clustering_model = AgglomerativeClustering(
    n_clusters=chosen_k,
    linkage='complete',
    metric='precomputed'
)
    
cluster_labels = clustering_model.fit_predict(condensed_dist_matrix)

score = silhouette_score(
    condensed_dist_matrix,
    cluster_labels,
    metric='precomputed'
)

print(f"Optimal number of clusters {chosen_k} suggested by Silhouette Score: {score}")

Optimal number of clusters 94 suggested by Silhouette Score: 0.6109787997712278


In [107]:
df_community_top_services_filtered_agg = df_community_top_services_filtered.groupby('Community').apply(lambda x: set(x.Service)).reset_index(name='Services')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6044\4263956792.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_community_top_services_filtered_agg = df_community_top_services_filtered.groupby('Community').apply(lambda x: set(x.Service)).reset_index(name='Services')


In [108]:
df_community_top_services_filtered_agg

,Community,Services
0,0,"{twitter, flickr, blog}"
1,1,"{twitter, feed, blog}"
2,2,"{twitter, feed, blog}"
3,3,"{youtube, twitter, blog}"
4,4,"{twitter, feed, blog}"
...,...,...
223,257,"{youtube, flickr, blog}"
224,259,"{flickr, delicious, feed}"
225,260,"{youtube, googletalk}"
226,261,"{picasa, twitter, googletalk}"


In [109]:
df_community_top_services_filtered_agg['Cluster'] = cluster_labels

In [110]:
df_community_top_services_filtered_agg

,Community,Services,Cluster
0,0,"{twitter, flickr, blog}",21
1,1,"{twitter, feed, blog}",17
2,2,"{twitter, feed, blog}",17
3,3,"{youtube, twitter, blog}",38
4,4,"{twitter, feed, blog}",17
...,...,...,...
223,257,"{youtube, flickr, blog}",12
224,259,"{flickr, delicious, feed}",23
225,260,"{youtube, googletalk}",7
226,261,"{picasa, twitter, googletalk}",41


In [111]:
df_community_cluster_features = df_community_top_services_filtered_agg.explode('Services').groupby(['Cluster', 'Services']).mean().reset_index().rename({ 'Community': 'Mean' }, axis=1).sort_values(['Cluster', 'Mean'], ascending=[True, False])

In [112]:
df_community_cluster_features

,Cluster,Services,Mean
0,0,flickr,245.0
1,0,googletalk,245.0
2,0,youtube,227.0
4,1,facebook,180.0
5,1,flickr,180.0
...,...,...,...
262,92,linkedin,26.0
263,92,twitter,26.0
264,93,blog,23.0
265,93,digg,23.0


In [113]:
top_feature_count = 3
cluster_summaries = []

for cluster_id in range(chosen_k):

    top_features = df_community_cluster_features[df_community_cluster_features['Cluster'] == cluster_id]['Services'].head(top_feature_count)

    cluster_summaries.append({
        'Cluster': cluster_id,
        'Top Services': set(top_features),
        'Communitiy_Count': len(df_community_top_services_filtered_agg[df_community_top_services_filtered_agg['Cluster'] == cluster_id])
    })


cluster_profiles = pd.DataFrame(cluster_summaries)

In [114]:
cluster_profiles.sort_values('Communitiy_Count', ascending=False).head(5)

,Cluster,Top Services,Communitiy_Count
20,20,{blog},22
29,29,{twitter},18
15,15,"{twitter, delicious, blog}",17
21,21,"{twitter, flickr, blog}",11
17,17,"{twitter, feed, blog}",11


In [115]:
cluster_profiles.sort_values('Communitiy_Count', ascending=True).head(5)

,Cluster,Top Services,Communitiy_Count
25,25,"{feed, delicious, digg}",1
26,26,"{brightkite, backtype, digg}",1
30,30,"{twitter, linkedin, blog}",1
31,31,"{lastfm, facebook, blog}",1
23,23,"{flickr, delicious, feed}",1


In [116]:
len(set(df_community_cluster_features['Services'].tolist()))

30

In [117]:
service_interpretation = {
    # Mainstream Social Media
    'facebook': 'Mainstream Social Media',
    'twitter': 'Mainstream Social Media',
    'youtube': 'Mainstream Social Media',
    
    # Blogging / Content Sharing
    'blog': 'Blogging ',
    'tumblr': 'Blogging ',
    'livejournal': 'Blogging',
    'plurk': 'Blogging',
    'backtype': 'Blogging',
    
    # Social Bookmarking / Curation
    'delicious': 'Social Bookmarking ',
    'digg': 'Social Bookmarking ',
    'diigo': 'Social Bookmarking',
    'misterwong': 'Social Bookmarking',
    'stumbleupon': 'Social Bookmarking ',
    
    # Professional / Networking
    'linkedin': 'Professional',
    'googletalk': 'Professional ',
    'identica': 'Professional',
    
    # Entertainment / Media
    'lastfm': 'Entertainment',
    'netflix': 'Entertainment ',
    'pandora': 'Entertainment ',
    'mixx': 'Entertainment',
    
    # Photography / Visual Sharing
    'flickr': 'Photography ',
    'picasa': 'Photography',
    'photobucket': 'Photography ',
    
    # Feed / Reader Platforms
    'feed': 'Feed ',
    'googlereader': 'Feed ',
    'googleshared': 'Feed',
    
    # Other / Niche
    'amazon': 'Other',
    'ameba': 'Other',
    'brightkite': 'Other',
    'disqus': 'Other',
}

In [118]:
def get_cluster_interpretations(top_services):
    return list({service_interpretation.get(s, 'Other / Misc') for s in top_services})

In [119]:
cluster_profiles['Interpretation'] = cluster_profiles['Top Services'].apply(get_cluster_interpretations)
cluster_profiles

,Cluster,Top Services,Communitiy_Count,Interpretation
0,0,"{youtube, flickr, googletalk}",2,"[Photography , Mainstream Social Media, Profes..."
1,1,"{flickr, facebook, blog}",3,"[Photography , Mainstream Social Media, Bloggi..."
2,2,"{picasa, feed, facebook}",2,"[Photography, Mainstream Social Media, Feed ]"
3,3,"{picasa, pandora, googlereader}",2,"[Photography, Feed , Entertainment ]"
4,4,"{youtube, delicious, blog}",2,"[Mainstream Social Media, Social Bookmarking ,..."
...,...,...,...,...
89,89,"{twitter, lastfm, flickr}",1,"[Photography , Mainstream Social Media, Entert..."
90,90,"{ameba, twitter}",1,"[Mainstream Social Media, Other]"
91,91,"{livejournal, feed, blog}",1,"[Blogging, Blogging , Feed ]"
92,92,"{twitter, digg, linkedin}",1,"[Mainstream Social Media, Social Bookmarking ,..."


In [121]:
cluster_profiles.sort_values('Communitiy_Count', ascending=True)

,Cluster,Top Services,Communitiy_Count,Interpretation
25,25,"{feed, delicious, digg}",1,"[Social Bookmarking , Feed ]"
26,26,"{brightkite, backtype, digg}",1,"[Blogging, Social Bookmarking , Other]"
30,30,"{twitter, linkedin, blog}",1,"[Mainstream Social Media, Blogging , Professio..."
31,31,"{lastfm, facebook, blog}",1,"[Mainstream Social Media, Blogging , Entertain..."
23,23,"{flickr, delicious, feed}",1,"[Photography , Social Bookmarking , Feed ]"
...,...,...,...,...
17,17,"{twitter, feed, blog}",11,"[Mainstream Social Media, Blogging , Feed ]"
21,21,"{twitter, flickr, blog}",11,"[Photography , Mainstream Social Media, Bloggi..."
15,15,"{twitter, delicious, blog}",17,"[Mainstream Social Media, Social Bookmarking ,..."
29,29,{twitter},18,[Mainstream Social Media]


In [122]:
cluster_profiles.describe()

,Cluster,Communitiy_Count
count,94.000000,94.000000
mean,46.500000,2.425532
std,27.279418,3.611479
min,0.000000,1.000000
25%,23.250000,1.000000
50%,46.500000,1.000000
75%,69.750000,2.000000
max,93.000000,22.000000
